# **Comparative Analysis of Performance Ratios for Optimal Portfolio Selection**
<p>Portfolio optimization is a cornerstone of modern investment management, aiming to maximize returns while minimizing risk. This program explores various portfolio optimization strategies, including equal-weighted, minimum volatility, maximum Sharpe ratio, risk parity, momentum, and Monte Carlo simulation. Using historical data from ten major stocks, we evaluate the performance of these strategies based on key metrics such as annualized return, volatility, Sharpe ratio, Sortino ratio, and other risk-adjusted measures.</p>

### <strong><u>Loading Important Libraries</u></strong>:


<p><span style="background-color: #E8E8E8"><strong>"numpy"</strong></span>: <i>numpy</i> is a library used for mathematical operations on arrays and matrices, providing support for mathematical functions and operations that are optimized for numerical computing. </p>

<p><span style="background-color: #E8E8E8"><strong>"pandas"</strong></span>: <i>pandas</i> is a data manipulation and analysis library that provides data structures (such as DataFrame) and functions to work with structured data, enabling data cleaning, wrangling, and analysis tasks.</p>

<p><span style="background-color: #E8E8E8"><strong>"yfinance"</strong></span>: <i>yfinance</i> is a popular Python library used for retrieving financial data from Yahoo Finance.  It's a powerful and convenient tool for anyone working with stock market data, as it provides a simple interface to access a wide range of information. It is essential for tasks like backtesting trading strategies, building financial models, performing technical analysis and portfolio optimization.</p>

<p><span style="background-color: #E8E8E8"><strong>"scipy.optimize"</strong></span>: This module provides a powerful collection of tools for optimization, which is the process of finding the best solution to a problem – often meaning finding the minimum or maximum value of a function. The <i>minimize</i> function in Python is a powerful tool for finding the minimum of a scalar function of one or more variables. It's part of the SciPy (Scientific Python) library, which provides a wide range of scientific computing tools.  In the context of portfolio optimization, minimize is commonly used to find the portfolio weights that minimize a specific objective function, such as portfolio variance or other risk measures.</p>

<p><span style="background-color: #E8E8E8"><strong>"tabulate"</strong></span>: The <i>tabulate</i> library in Python is a fantastic tool for creating nicely formatted tables in your terminal or output.  It's particularly useful for presenting data clearly and readably, especially when dealing with tabular data that you want to display in a structured way.</p>

In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import scipy.optimize as sco
from tabulate import tabulate

### <strong><u>Constants</u></strong>:
<p>Our sample portfolio consists of 10 stocks:<i>"AAPL", "MSFT", "GOOGL", "TSLA", "JNJ", "PFE", "XOM", "CVX", "WMT", "PG"</i> and is evaluated over a period of 5 years (<i>01/01/2020 - 01/01/2025</i>).</p>
<p>The Risk-Free Rate was approximated by using the 10-year US Treasury Note, which was found at: https://www.cnbc.com/quotes/US10Y. On February the 14th in 2025, it was roughly at 4.527%.</p>

In [5]:
TICKERS = ["MSFT", "AAPL", "GOOGL", "TSLA", "JNJ", "PFE", "XOM", "CVX", "WMT", "PG"]
START_DATE = "2020-01-01"
END_DATE = "2025-01-01"
RISK_FREE_RATE = 0.05 # ~ 4.527% (https://www.cnbc.com/quotes/US10Y) approximated by the 10-year US Treasury Note.

# For the Momentum Portfolio:
MOMENTUM_PERIOD = 126  # 6 months of trading days.
TOP_N = 5  # Number of stocks to include in the momentum portfolio.

# For the Monte Carlo Portfolio:
NUM_PORTFOLIOS = 10000  # Number of random portfolios to simulate.

### <strong><u>Fetch Historical Prices and Calculate Returns</u></strong>:

In [7]:
def fetch_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date, progress=False)["Adj Close"]
    returns = data.pct_change().dropna()
    mean_returns = returns.mean() * 252   # Annualized.
    cov_matrix = returns.cov() * 252   # Annualized.
    return returns, mean_returns, cov_matrix

In [8]:
# Portfolio volatility function
def portfolio_volatility(weights, *args):
    cov_matrix = args[0]
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# Negative Sharpe ratio function (for minimization)
def neg_sharpe_ratio(weights, *args):
    mean_returns, cov_matrix, risk_free_rate = args
    port_return = np.dot(weights, mean_returns)
    port_volatility = portfolio_volatility(weights, cov_matrix)
    return -((port_return - risk_free_rate) / port_volatility)

# Risk parity objective function
def risk_parity_objective(weights, *args):
    cov_matrix = args[0]
    port_volatility = portfolio_volatility(weights, cov_matrix)
    risk_contributions = weights * np.dot(cov_matrix, weights) / port_volatility
    return np.sum((risk_contributions - np.ones_like(weights) / len(weights)) ** 2)

# Basic portfolio metrics
def calculate_basic_metrics(weights, mean_returns, cov_matrix, risk_free_rate):
    port_return = np.dot(weights, mean_returns)
    port_volatility = portfolio_volatility(weights, cov_matrix)
    sharpe_ratio = (port_return - risk_free_rate) / port_volatility
    return port_return, port_volatility, sharpe_ratio

# Additional portfolio metrics
def calculate_additional_metrics(weights, returns, risk_free_rate, market_returns, benchmark_returns):
    port_returns = np.dot(returns, weights)
    downside_returns = port_returns[port_returns < 0]
    downside_std = downside_returns.std() * np.sqrt(252)
    sortino_ratio = (np.mean(port_returns) * 252 - risk_free_rate) / downside_std

    excess_returns = port_returns - risk_free_rate
    omega_ratio = np.mean(np.maximum(excess_returns, 0)) / np.mean(np.maximum(-excess_returns, 0))

    beta = np.cov(port_returns, market_returns)[0, 1] / np.var(market_returns)
    treynor_ratio = (np.mean(port_returns) * 252 - risk_free_rate) / beta

    excess_benchmark_returns = port_returns - benchmark_returns
    information_ratio = np.mean(excess_benchmark_returns) * 252 / (np.std(excess_benchmark_returns) * np.sqrt(252))

    cumulative_returns = pd.Series((1 + port_returns).cumprod())
    drawdowns = (cumulative_returns / cumulative_returns.cummax() - 1)
    ulcer_index = np.sqrt(np.mean(drawdowns ** 2))

    average_drawdown = np.mean(drawdowns)
    sterling_ratio = (np.mean(port_returns) * 252 - risk_free_rate) / abs(average_drawdown)

    return {
        "sortino_ratio": sortino_ratio,
        "omega_ratio": omega_ratio,
        "treynor_ratio": treynor_ratio,
        "information_ratio": information_ratio,
        "ulcer_index": ulcer_index,
        "sterling_ratio": sterling_ratio
    }

# Optimize portfolio
def optimize_portfolio(mean_returns, cov_matrix, risk_free_rate, objective_function, bounds, constraints, init_guess):
    args = (mean_returns, cov_matrix, risk_free_rate) if objective_function == neg_sharpe_ratio else (cov_matrix,)
    opt_result = sco.minimize(objective_function, init_guess, args=args,
                              method='SLSQP', bounds=bounds, constraints=constraints)
    return opt_result.x

# Main function
def main():
    returns, mean_returns, cov_matrix = fetch_data(TICKERS, START_DATE, END_DATE)
    market_returns = yf.download("^GSPC", start=START_DATE, end=END_DATE, progress=False)["Adj Close"].pct_change().dropna()
    benchmark_returns = market_returns

    # Constraints and bounds for optimization
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
    bounds = [(0, 1)] * len(TICKERS)
    init_guess = np.ones(len(TICKERS)) / len(TICKERS)

    # Equal-weighted portfolio
    eq_weights = np.ones(len(TICKERS)) / len(TICKERS)
    eq_return, eq_volatility, eq_sharpe = calculate_basic_metrics(eq_weights, mean_returns, cov_matrix, RISK_FREE_RATE)
    eq_additional = calculate_additional_metrics(eq_weights, returns, RISK_FREE_RATE, market_returns, benchmark_returns)

    # Minimum Volatility Portfolio
    mv_weights = optimize_portfolio(mean_returns, cov_matrix, RISK_FREE_RATE, portfolio_volatility, bounds, constraints, init_guess)
    mv_return, mv_volatility, mv_sharpe = calculate_basic_metrics(mv_weights, mean_returns, cov_matrix, RISK_FREE_RATE)
    mv_additional = calculate_additional_metrics(mv_weights, returns, RISK_FREE_RATE, market_returns, benchmark_returns)

    # Maximum Sharpe Ratio Portfolio
    ms_weights = optimize_portfolio(mean_returns, cov_matrix, RISK_FREE_RATE, neg_sharpe_ratio, bounds, constraints, init_guess)
    ms_return, ms_volatility, ms_sharpe = calculate_basic_metrics(ms_weights, mean_returns, cov_matrix, RISK_FREE_RATE)
    ms_additional = calculate_additional_metrics(ms_weights, returns, RISK_FREE_RATE, market_returns, benchmark_returns)

    # Risk Parity Portfolio
    rp_weights = optimize_portfolio(mean_returns, cov_matrix, RISK_FREE_RATE, risk_parity_objective, bounds, constraints, init_guess)
    rp_return, rp_volatility, rp_sharpe = calculate_basic_metrics(rp_weights, mean_returns, cov_matrix, RISK_FREE_RATE)
    rp_additional = calculate_additional_metrics(rp_weights, returns, RISK_FREE_RATE, market_returns, benchmark_returns)

    # Momentum Portfolio
    momentum = returns.tail(MOMENTUM_PERIOD).add(1).prod() - 1
    momentum_rank = momentum.rank(ascending=False)
    momentum_weights = np.zeros(len(TICKERS))
    momentum_weights[momentum_rank <= TOP_N] = 1 / TOP_N
    momentum_return, momentum_volatility, momentum_sharpe = calculate_basic_metrics(momentum_weights, mean_returns, cov_matrix, RISK_FREE_RATE)
    momentum_additional = calculate_additional_metrics(momentum_weights, returns, RISK_FREE_RATE, market_returns, benchmark_returns)

    # Monte Carlo Simulation
    results = np.zeros((3, NUM_PORTFOLIOS))
    weights_record = np.zeros((NUM_PORTFOLIOS, len(TICKERS)))
    for i in range(NUM_PORTFOLIOS):
        weights = np.random.random(len(TICKERS))
        weights /= np.sum(weights)
        port_return, port_volatility, port_sharpe = calculate_basic_metrics(weights, mean_returns, cov_matrix, RISK_FREE_RATE)
        results[0, i] = port_return
        results[1, i] = port_volatility
        results[2, i] = port_sharpe
        weights_record[i, :] = weights
    max_sharpe_idx = np.argmax(results[2])
    mc_weights = weights_record[max_sharpe_idx, :]
    mc_return, mc_volatility, mc_sharpe = calculate_basic_metrics(mc_weights, mean_returns, cov_matrix, RISK_FREE_RATE)
    mc_additional = calculate_additional_metrics(mc_weights, returns, RISK_FREE_RATE, market_returns, benchmark_returns)

    # Compile portfolio metrics
    portfolios = {
        "Equal-weighted": (eq_return, eq_volatility, eq_sharpe, eq_additional, eq_weights),
        "Min Volatility": (mv_return, mv_volatility, mv_sharpe, mv_additional, mv_weights),
        "Max Sharpe": (ms_return, ms_volatility, ms_sharpe, ms_additional, ms_weights),
        "Risk Parity": (rp_return, rp_volatility, rp_sharpe, rp_additional, rp_weights),
        "Momentum": (momentum_return, momentum_volatility, momentum_sharpe, momentum_additional, momentum_weights),
        "Monte Carlo": (mc_return, mc_volatility, mc_sharpe, mc_additional, mc_weights)
    }

    # Display results
    performance = []
    for name, (ret, vol, sharpe, additional, weights) in portfolios.items():
        performance.append([
            name,
            round(ret, 4),
            round(vol, 4),
            round(sharpe, 4),
            round(additional["sortino_ratio"], 4),
            round(additional["omega_ratio"], 4),
            round(additional["treynor_ratio"], 4),
            round(additional["information_ratio"], 4),
            round(additional["ulcer_index"], 4),
            round(additional["sterling_ratio"], 4)
        ])

    portfolio_results = pd.DataFrame(performance, columns=[
        "Portfolio", "Annual\nReturn", "Volatility", "Sharpe\nRatio", 
        "Sortino\nRatio", "Omega\nRatio", "Treynor\nRatio", 
        "Information\nRatio", "Ulcer\nIndex", "Sterling\nRatio"
    ])
    print("Portfolio Performance Metrics:")
    print(tabulate(portfolio_results, headers='keys', tablefmt='grid'))

    # Display optimal weights
    weights_df = pd.DataFrame({
        "Ticker": TICKERS,
        "Equal-weighted": eq_weights,
        "Min Volatility": mv_weights,
        "Max Sharpe": ms_weights,
        "Risk Parity": rp_weights,
        "Momentum": momentum_weights,
        "Monte Carlo": mc_weights
    })
    weights_df = weights_df.round(4)
    print("\nOptimal Weights for Each Portfolio:")
    print(tabulate(weights_df, headers='keys', tablefmt='grid', showindex=False))

if __name__ == "__main__":
    main()

Portfolio Performance Metrics:
+----+----------------+----------+--------------+----------+-----------+---------+-----------+---------------+---------+------------+
|    | Portfolio      |   Annual |   Volatility |   Sharpe |   Sortino |   Omega |   Treynor |   Information |   Ulcer |   Sterling |
|    |                |   Return |              |    Ratio |     Ratio |   Ratio |     Ratio |         Ratio |   Index |      Ratio |
+====+================+==========+==============+==========+===========+=========+===========+===============+=========+============+
|  0 | Equal-weighted |   0.2267 |       0.2066 |   0.8554 |    1.0425 |  0.0017 |    0.198  |        1.0198 |  0.0639 |     4.1592 |
+----+----------------+----------+--------------+----------+-----------+---------+-----------+---------------+---------+------------+
|  1 | Min Volatility |   0.1289 |       0.1649 |   0.4782 |    0.6195 |  0.0013 |    0.1366 |       -0.0878 |  0.0483 |     2.3131 |
+----+----------------+--------